In [ ]:
### 1. Using the amount of positive words and negative words to measure
### 2. Improvement: considering the effect of  Negative Words and Gradable Adverbs words
import pandas as pd

#pd.read_csv(file) is named as A1 which we use a lot later
A1 = pd.read_csv('WomensApparelReviews(1).csv')

#delete rows with blank titles or reviews
A1.dropna(axis=0, how='any', inplace=True)

#turn positive-words into pw list
pw0 = []
#turn the txt file into list
with open('positive-words.txt','r') as f1:
    for line in f1:
        pw0.append(list(line.strip('\n').split(',')))
#the above list is a nested list 
#get a normal list by dropping '[' and ']'
pw = eval('[%s]'%repr(pw0).replace('[', '').replace(']', ''))

#turn negative-words(1) into nw list, same method
nw0 = []
with open('negative-words(1).txt','r') as f2:
    for line in f2:
        nw0.append(list(line.strip('\n').split(',')))
nw = eval('[%s]'%repr(nw0).replace('[', '').replace(']', ''))

#turn NegatingWordList into notw list, same method
notw0 = []
with open('NegatingWordList.txt','r') as f3:
    for line in f3:
        notw0.append(list(line.strip('\n').split(',')))
notw = eval('[%s]'%repr(notw0).replace('[', '').replace(']', ''))

#turn adv-words into advw list, same method
advw0 = []
with open('adv-words.txt','r') as f4:
    for line in f4:
        advw0.append(list(line.strip('\n').split(',')))
advw = eval('[%s]'%repr(advw0).replace('[', '').replace(']', ''))

#divide the gradable adverbs words into strong emotion, meadium emotion and weak emotion
#this is previously sorted when we create the adv-words txt
s_advw = advw[:23]   
m_advw = advw[23:46]
w_advw = advw[46:55]

#set weights for different gradable adverbs
w1 = 3
w2 = 2
w3 = 1
w4 = 0.5

#the improved function to get each text's emotion mark
def whatpoint(a,b):
    '''
    the function analyzes positive and negative words and the negating and gradable adverbs 
    ahead of them in each text to get the emotion mark
    the input a and b are to locate a text in the WomensApparelReviews file
    this function returns the emotion mark of one text
    '''
    
    t = A1.iloc[a,b]
    #standardize the sentence by lowering and droping symbols 
    t = t.lower()
    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace('!','')
    t = t.replace('-','')
    t = t.replace(';','')
    t = t.replace(':','')
    t = t.replace('?','')
    t = t.replace(')','')
    t = t.replace('(','')
    t = t.replace('/','')
    #break the sentence into seperate word
    ff = t.split()
    
    #ppoint is total positive emotion mark
    #npoint is total negative emotion mark
    ppoint = 0
    npoint = 0
    for i in range(len(ff)):
        #judge whether the text is only 1 word
        if len(ff) == 1:
            if ff[i] in pw:
                ppoint += 1
            elif ff[i] in nw:
                npoint += 1
        #if not only one word, deduct whether each word is positive or negative        
        elif len(ff) != 1:
            #then dedect the word before the positive word is whether gradable word or not
            #and in which gradable adv word list for later assigning weights
            if ff[i] in pw:
                #then dedect the word before the adv is whether negating word or not
                #then set the weights
                if ff[i-1] in s_advw:
                    if ff[i-2] in notw:
                        npoint += w3 * 1
                    else:
                        ppoint += w1 * 1
                elif ff[i-1] in m_advw:
                    if ff[i-2] in notw:
                        npoint += w3 * 1
                    else:
                        ppoint += w2 * 1         
                elif ff[i-1] in w_advw:
                    if ff[i-2] in notw:
                        npoint += w4 * 1
                    else:
                        ppoint += w4 * 1
                elif ff[i-1] in notw:
                    npoint += w3 * 1
                else:
                    ppoint += w3 * 1
            
            #same method above, but change the condition into negative words   
            elif ff[i] in nw:
                if ff[i-1] in s_advw:
                    if ff[i-2] in notw:
                        ppoint += w3 * 1
                    else:
                        npoint += w1 * 1
                elif ff[i-1] in m_advw:
                    if ff[i-2] in notw:
                        ppoint += w3 * 1
                    else:
                        npoint += w2 * 1         
                elif ff[i-1] in w_advw:
                    if ff[i-2] in notw:
                        ppoint += w4 * 1
                    else:
                        npoint += w4 * 1
                elif ff[i-1] in notw:
                    ppoint += w3 * 1
                else:
                    npoint += w3 * 1
    
    #positive point minus negative point to get the final emotion mark
    return (ppoint-npoint)

#this function counts the amount of positive/negative words
def howmany(a,b):
    '''
    this function count the positive and negative words in each text
    the input a and b are to locate a text in the WomensApparelReviews file
    the output is the amount of positive words minus the amount of negative words
    
    '''
    
    t = A1.iloc[a,b]
    #standardize the sentence by lowering and droping symbols
    t = t.lower()
    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace('!','')
    t = t.replace('-','')
    t = t.replace(';','')
    t = t.replace(':','')
    t = t.replace('?','')
    t = t.replace(')','')
    t = t.replace('(','')
    t = t.replace('/','')
    #break the sentence into seperate word
    ff = t.split()
    
    #numberpw is total amount of positive words in each text
    #numbernw is total amount of negative words in each text
    numberpw = 0
    numbernw = 0
    
    for i in ff:
        if i in pw:
            numberpw += 1
   
        elif i in nw:
            numbernw += 1       
    
    #positive word amount minus negative word amount to get the final emotion mark           
    return (numberpw - numbernw)

point_title_list = []
number_title_list = []
point_comment_list = []
number_comment_list = []

#get all title's emotion mark by howmany()
#then set this list as a new column
for i in range(len(A1)):
    c = howmany(i,3)
    number_title_list.append(c)  
A1['number_title'] = number_title_list 

#get all title's emotion mark by whatpoint()
#then set this list as a new column
for i in range(len(A1)):
    c = whatpoint(i,3)
    point_title_list.append(c)    
A1['point_title'] = point_title_list

#get all reviews's emotion mark by howmany()
#then set this list as a new column
for i in range(len(A1)):
    c = howmany(i,4)
    number_comment_list.append(c)    
A1['number_comment'] = number_comment_list

#get all reviews's emotion mark by whatpoint()
#then set this list as a new column
for i in range(len(A1)):
    c = whatpoint(i,4)
    point_comment_list.append(c)
A1['point_comment'] = point_comment_list

In [ ]:
display(A1)

In [ ]:
### 3. Sales volume by department
import numpy as np
import matplotlib.pyplot as plt

#count the sales amount by department
review = pd.read_csv('WomensApparelReviews(1).csv')
salevolume_department = review.groupby("Department")["Product ID"].count()
print (salevolume_department)
#drew the pie picture of 'Sales volume by department'
salevolume_department.index
names = [u'Intimate',u'Dresses',u'Bottoms',u'Tops',u'Jackets',u'Trend']
colors = ['coral','orangered','saddlebrown','chocolate','goldenrod','peachpuff']
plt.pie(salevolume_department,labels=names,colors=colors,autopct='%1.1f%%')
plt.title(u'Sales volume by department')
plt.figure(figsize=(6,6))
plt.show()

In [ ]:
### 4. Top five products with the most sales
#get the Top five products with the most sales
#use 'Product ID' to class
new_review = review.groupby('Product ID').count()
#sort the above values to get the best selling products
most_popular_products = new_review.sort_values('Unnamed: 0', ascending=False)['Unnamed: 0'].head(5)
print(most_popular_products)   

#drew the bar picture for Top five products with the most sales
index = np.arange(1,6)
plt.bar(index,most_popular_products,0.5,color='darkseagreen')
plt.xticks(index,[u'1078',u'862',u'1094',u'1081',u'872'])
plt.xlabel(u'Product ID')
plt.ylabel(u'Sales volume')
plt.title(u'Top five products with the most sales')
plt.show()

In [ ]:
### 5. The highest and lowest rating product
#Get the highest rated product
#use 'Product ID' to class and then calculate the average of Raing
product_rating = review.groupby("Product ID")["Rating"].mean().reset_index(name="mean_rating")
#sort above values
highest_rated_products = product_rating.sort_values('mean_rating', ascending=False )['mean_rating'].head(400)

In [ ]:
display(highest_rated_products)

In [ ]:
#Get the worst rated product

worst_rated_products = product_rating.sort_values('mean_rating', ascending=True)['mean_rating'].head(21)
display(worst_rated_products)

In [ ]:
### 6. What is the age distribution of customers?
#catalogue age by group
bins = [-1,12,20,30,45,130]
labels_age = [u'0-12 years',u'13-20 years',u'21-30 years',u'31-45 years',u'46 years+']
#set the age group as a new column
#then use 'Age grouping' to class and then count how many deals in total
review['Age grouping'] = pd.cut(review['Age'],bins,labels=labels_age)
agegroup = review.groupby('Age grouping')['Product ID'].count()
print(agegroup)


#drew the bar picture of 'Number of people of all age groups'
index = np.arange(1,6)
plt.bar(index,agegroup,0.5,color='darkseagreen')
plt.xticks(index,[u'0-12 years',u'13-20 years',u'21-30 years',u'31-45 years',u'46 years+'])
plt.xlabel(u'Age group')
plt.ylabel(u'Number of people')
plt.title(u'Number of people of all age groups')
plt.legend([u'number of people',],loc='upper right')
plt.show()

In [ ]:
### 7. What is the most popular item in each age group?
#Most popular products of every age group

#use both 'Age grouping' and 'Product ID' to class and then count the 'Product ID'
age_id_count = review.groupby(['Age grouping', 'Product ID'])['Product ID'].count().reset_index(name="count")

#use 'Age grouping' to class and then find the max of the above result
age_id_max = age_id_count.groupby(['Age grouping'])['count'].max().reset_index(name="count")

pd.merge(age_id_max, age_id_count, on=['Age grouping', 'count'],how='left')

In [ ]:
### 8. Are older people more inclined to recommend a product or younger people?
#Get the relationship between recommendation and age
age_recommendation = review.groupby('Is it Recommended?')['Age'].mean()
print (age_recommendation)
#drew the bar picture for the relationship between recommendation and age
index = np.arange(1,3)
plt.bar(index,age_recommendation,0.3,color='darkseagreen')
plt.xticks(index,[u'Not recommended',u'Recommend'])
plt.xlabel(u'Is it Recommended?')
plt.ylabel(u'Age')
plt.title(u'The relationship between recommendation and age')
plt.show()


In [ ]:
### 9. Is the rating correlated to your measure of negativity-positivity?
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

#draw the graph of 'point_comment' and 'Rating'
sns.pairplot(A1, x_vars='Rating', y_vars='point_comment', height=5, aspect=0.8, kind='reg')

display(A1.corr())

#apply OLS
x1 = A1['Rating']
y1 = A1['point_comment']
x1 = sm.add_constant(x1)

est1 = sm.OLS(y1,x1)
est1 = est1.fit()
est1.summary()

In [ ]:
### 10. Which group of reviewers wrote a longer text in their review?
#get the length of each review
length_comments = []
for i in range(len(A1)):
    #standarize the text
    t = A1.iloc[i,4]
    t = t.lower()
    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace('!','')
    t = t.replace('-','')
    t = t.replace(';','')
    t = t.replace(':','')
    t = t.replace('?','')
    t = t.replace(')','')
    t = t.replace('(','')
    t = t.replace('/','')
    ff = t.split()
    
    aa = len(ff)
    length_comments.append(aa)

A1['length_comments'] = length_comments

#draw the relationship
sns.pairplot(A1, x_vars='point_comment', y_vars='length_comments', height=5, aspect=0.8, kind='reg')

#apply OLS to analyze
x2 = A1['point_comment']
y2 = A1['length_comments']
x2 = sm.add_constant(x2)

est2 = sm.OLS(y2,x2)
est2 = est2.fit()
est2.summary()

In [ ]:
### 11. The relationship between Rating and Recommendation
#draw the graph and the regression line
sns.pairplot(A1, x_vars='Rating', y_vars='Is it Recommended?', height=5, aspect=0.8, kind='reg')

#apply OLS to analyze
x3 = A1['Rating']
y3 = A1['Is it Recommended?']
x3 = sm.add_constant(x3)

est3 = sm.OLS(y3,x3)
est3 = est3.fit()
est3.summary()

In [ ]:
### 12. The relationship between Age and Rating
#get the relationship between age and rating
x_age = review['Age']
y_rating = review['Rating']
plt.scatter(x_age,y_rating,s=1)
plt.title("Correlation of age and rating",fontsize=14)
plt.xlabel("Age",fontsize=14)
plt.ylabel("Rating",fontsize=14)
plt.tick_params(axis='both',which='major',labelsize=14)
plt.show()


In [ ]:
### 13. Which age group uses more positive words? Which age group uses more negative words?
#get the average number of positive/negative words used in different age groups
A1['Age group'] = review.iloc[:,8]
AgeGroup_NumberComment_mean = A1.groupby('Age group')['number_comment'].mean()
print (AgeGroup_NumberComment_mean)

#drew the bar picture of 'Number of positive/negative words used in different age groups'
index = np.arange(1,6)
plt.bar(index,AgeGroup_NumberComment_mean,0.5,color='darkseagreen')
plt.xticks(index,[u'0-12 years',u'13-20 years',u'21-30 years',u'31-45 years',u'46 years+'])
plt.xlabel(u'Age group')
plt.ylabel(u'Number of positive/negative words in comments')
plt.title(u'Number of positive/negative words used in different age groups')
plt.show()

In [ ]:
### 14. Is it true that unhappy customers use more capital letters? or it is the other way around?
#get the number of uppercase letters in the comments
import re 
Number_UppercaseLetters = []

#first get all the capial letters in a sentence
#then use len() to get the length and add into a new list 
for i in range(len(A1)):
    dd = A1.iloc[i,4]
    UppercaseLetters = re.findall(r'[A-Z]',dd)
    Number_UppercaseLetters.append(len(UppercaseLetters))

#turn the list into a column to do further analysis
A1['Number_UppercaseLetters'] = Number_UppercaseLetters

#drew the picture of 'the relationship between the number of uppercase letters in the comments and satisfaction'
sns.pairplot(A1, x_vars='point_comment', y_vars='Number_UppercaseLetters', height=5, aspect=0.8)

#do the OLS
x4 = A1['point_comment']
y4 = A1['Number_UppercaseLetters']
x4 = sm.add_constant(x4)

est4 = sm.OLS(y4,x4)
est4 = est4.fit()
est4.summary()

In [ ]:
### 15. What is the average rating in positive, negative or neutral reviews?
#average points for all the positive and negative reviews

simple_measure = []

# Based on the number of measurement for review text to set simple_measure=1/0/-1
for i in range(len(A1)):
    z = A1.iloc[i,11]
    if z > 0:
        simple_measure.append(1)
    elif z == 0:
        simple_measure.append(0)
    elif z < 0:
        simple_measure.append(-1)

# write this list into the dataframe       
A1['simple_measure'] = simple_measure

# class by simple_measure, then get the average of the Rating
A1.groupby('simple_measure')['Rating'].mean()

In [ ]:
### 16. Are there many outliers who wrote a negative text but left a high rating (or vice versa)?
#find the outliers

#count by setting high points and low rating
liar_lowrating = 0
for i in range(len(A1)):
    if A1.iloc[i,11] > 0 and A1.iloc[i,5] < 3:
        liar_lowrating += 1
print('positive liars with low rating:' , liar_lowrating)

#count by setting low points and high rating
liar_highrating = 0
for i in range(len(A1)):
    if A1.iloc[i,11] < 0 and A1.iloc[i,5] > 3:
        liar_highrating += 1
print('negative liars with high rating:' , liar_highrating)

In [ ]:
### 17. Which product attracted the most positive or negative reviews?
#which product has most positive reviews
#use both 'Product ID' and 'simple measure' to class and then count the number by suming the simple_measure
good1 = A1.groupby(['Product ID','simple_measure'])['simple_measure'].sum().reset_index(name='good_product')
#sort the above result to get the top 15 good products
good2 = good1.sort_values(by='good_product',ascending=False).head(15)
print('product with most positive reviews:')
print(good2)

#which product has most negative reviews
bad1 = A1.groupby(['Product ID','simple_measure'])['simple_measure'].sum().reset_index(name='bad_product')
#sort the above result to get the top 15 bad products
bad2 = bad1.sort_values(by='bad_product',ascending=True).head(15)
print('product with most negative reviews:')
print(bad2)